In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from scipy import stats

In [ ]:
data = pd.read_csv("dataset.csv")
data.head()

In [ ]:
# first two columns are unnecessary (index)
data.drop(data.columns[[0, 1]], axis = 1, inplace = True)
data.head()

In [ ]:
labels = ["radiation", "ozone", "temperature", "wind"]
fig, axs = plt.subplots(4, 4)
fig.set_size_inches(10, 10)
for i in range(4):
    for j in range(4):
        axs[i, j].axes.xaxis.set_visible(False)
        axs[i, j].axes.yaxis.set_visible(False)
        if i == j:
            axs[i, j].plot()
            axs[i, j].text(.5, .5, labels[i].capitalize(),  horizontalalignment='center', verticalalignment='center', transform=axs[i, j].transAxes, fontsize=13)
        else:
            if i == 0 or i == 3:
                axs[i, j].axes.xaxis.set_visible(True)
            if i == 0:
                axs[i, j].xaxis.tick_top()
            if j == 0 or j == 3:
                axs[i, j].axes.yaxis.set_visible(True)
            if j == 3:
                axs[i, j].yaxis.tick_right()
            axs[i, j].scatter(data[labels[j]], data[labels[i]], s=15, color="black")

In [ ]:
def tricubic(x):
        y = np.zeros_like(x)
        idx = (x >= -1) & (x <= 1)
        y[idx] = np.power(1.0 - np.power(np.abs(x[idx]), 3), 3)
        return y
plt.plot(np.linspace(-2, 2, 100), tricubic(np.linspace(-2, 2, 100)))

In [ ]:
class Loess:
    
    @staticmethod
    def standarize_data(data):
        data_mean = data.mean(0)
        data_std = data.std(0)
        return (data - data_mean) / data_std, data_mean, data_std

    def __init__(self, xx, yy):
        xx = np.array(xx)
        yy = np.array(yy)
        self.s_xx, self.mean_xx, self.std_xx = self.standarize_data(xx)
        self.s_yy, self.mean_yy, self.std_yy = self.standarize_data(yy)

    @staticmethod
    def get_min_range(distances, window):
        min_idx = np.argmin(distances)
        n = len(distances)
        if min_idx == 0:
            return np.arange(0, window)
        if min_idx == n-1:
            return np.arange(n - window, n)

        min_range = [min_idx]
        while len(min_range) < window:
            i0 = min_range[0]
            i1 = min_range[-1]
            if i0 == 0:
                min_range.append(i1 + 1)
            elif i1 == n-1:
                min_range.insert(0, i0 - 1)
            elif distances[i0-1] < distances[i1+1]:
                min_range.insert(0, i0 - 1)
            else:
                min_range.append(i1 + 1)
        return np.array(min_range)

    @staticmethod
    def get_weights(distances, min_range):
        max_distance = max(distances[min_range])
        weights = tricubic(distances[min_range] / max_distance)
        return weights

    def standarize_x(self, value):
        return (value - self.mean_xx) / self.std_xx

    def destandarize_y(self, value):
        return value * self.std_yy + self.mean_yy

    def estimate(self, x, f, d):
        window = int(len(self.s_xx) * f)
        s_x = self.standarize_x(x)

        distances = np.linalg.norm(self.s_xx - s_x, axis=1)

        #min_range = self.get_min_range(distances, window)
        min_range = np.argsort(distances)[:window]
        weights = self.get_weights(distances, min_range)
        wm = np.diag(weights)
        xm = self.s_xx[min_range]
        
        poly = PolynomialFeatures(degree=d)
        xm = poly.fit_transform(xm)
        xp = poly.fit_transform(s_x.reshape(1, -1))
        
        ym = self.s_yy[min_range]
        beta = np.linalg.pinv(xm.T @ wm @ xm) @ xm.T @ wm @ ym
        y = (xp @ beta)[0]
        return self.destandarize_y(y)[0]

In [ ]:
X = data[["radiation", "temperature", "wind"]]
Y = data[["ozone"]]
loess = Loess(X, Y)
estimations = []
for i in range(len(X)):
    x = np.array(X.iloc[[i]])[0]
    estimations.append(loess.estimate(x, 0.4, 1))
data["estimation"] = estimations
data["residual"] = data["ozone"] - data["estimation"]

In [ ]:
fig, ax = plt.subplots()
ax.set_box_aspect(1)
res = stats.probplot(np.sort(data["residual"]), plot=plt, fit=False)
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.set_box_aspect(1)
plt.scatter(data["estimation"], abs(data["residual"]), facecolors='none', edgecolors='black', s=20)
plt.xlabel("Fitted Values")
plt.ylabel("Absolute Residuals")

In [ ]:
fig, ax = plt.subplots()
ax.set_box_aspect(1)
plt.scatter(data["radiation"], data["residual"], facecolors='none', edgecolors='black', s=20)
plt.xlabel("Solar Radiation")
plt.ylabel("Residuals")

In [ ]:
fig, ax = plt.subplots()
ax.set_box_aspect(1)
plt.scatter(data["temperature"], data["residual"], facecolors='none', edgecolors='black', s=20)
plt.xlabel("Temperature")
plt.ylabel("Residuals")

In [ ]:
fig, ax = plt.subplots()
ax.set_box_aspect(1)
plt.scatter(data["wind"], data["residual"], facecolors='none', edgecolors='black', s=20)
plt.xlabel("Wind Speed")
plt.ylabel("Residuals")